In [ ]:
import numpy as np


import sys
import os
import io
import chardet
import codecs
import datetime

import cv2

from PIL import Image, ImageDraw

vgg_rgboffsets = [103.939, 116.779, 123.68]


image_fs = "rrc_t4/img/img_%d.jpg"
gt_fs = "rrc_t4/gt/gt_img_%d.txt"

#http://stackoverflow.com/questions/13590749/reading-unicode-file-data-with-bom-chars-in-python
def detect_encoding(fname):
    bytes = min(32, os.path.getsize(fname))
    raw = open(fname, 'rb').read(bytes)
    if raw.startswith(codecs.BOM_UTF8):
        encoding = 'utf-8-sig'
    else:
        result = chardet.detect(raw)
        encoding = result['encoding']
    return encoding



def process_groundtruth(gtfile,w=1280,h=720):
    image_match = Image.new('L',(w,h))
    image_dontcare = Image.new('L',(w,h))
    encoding = detect_encoding(gtfile)
    with io.open(gtfile,'r',encoding=encoding) as fd:
        for bbox_line in fd:
            fields = bbox_line.strip().split(',',8)
            #xvals = [int(x) for x in fields[:8:2]]
            #yvals = [int(y) for y in fields[1:8:2]]
            xycoords = [int(xy) for xy in fields[:-1]]
            label = fields[-1]
            draw = ImageDraw.Draw(image_dontcare if label=="###" else image_match)
            draw.polygon(xycoords,outline=None,fill=255)
    np_match = np.array(image_match,dtype=float)/255
    np_dontcare = np.array(image_dontcare,dtype=float)/255
    np_match_dc = np.maximum(np_match, np_dontcare)
    return(np_match,np_dontcare,np_match_dc)

def load_img(imgfile):
    img = Image.open(imgfile)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    iarray = np.array(img,dtype=float)
    for i in range(3):
        iarray[:,:,i] -= vgg_rgboffsets[i]
    return iarray #.transpose((2,0,1))

# Data loader
data in /dds/workspace/data_ja/

In [ ]:
!ls /dds/workspace/data_ja/

## Groundtruth

In [ ]:
import csv

In [ ]:
grountruth_path='/dds/workspace/data_ja/RF-CAT1-v1.0.csv'
groundtruth_csvfile = open(grountruth_path, 'r')
# with open(pathToGT, 'r') as csvfile:
groundtruth_reader = csv.DictReader(groundtruth_csvfile)#, delimiter=',')
groundtruth_dict = {}
for row in groundtruth_reader:
    groundtruth_dict.setdefault(row['filename'],[]).append(row)

## Images

In [ ]:
import argparse, magic, re, os, math, glob, shutil

In [ ]:
image_path='/dds/workspace/data_ja/CAT_1_9013.jpg'


In [ ]:
input_file=image_path

In [ ]:
t = magic.from_file(input_file)
try:
    if input_file.endswith('.jpg') or input_file.endswith('.jpeg'):
        dims = re.search(', (\d+)x(\d+)', t)
        width = int(dims.group(1))
        height = int(dims.group(2))
    elif input_file.endswith('.tif') or input_file.endswith('.tiff'):
        width = int(re.search('width=(\d+)', t).group(1))
        height = int(re.search('height=(\d+),', t).group(1))
    elif input_file.endswith('.png'):
        dims = re.search(', (\d+) x (\d+)', t)
        width = int(dims.group(1))
        height = int(dims.group(2))
    else:
        print('ERROR: Unknown source image type; JPG, TIFF, or PNG only! Quitting...')
except:
    print('ERROR: Could not parse source image dims! Quitting...')

In [ ]:
print('-> ' + str(width) + ' x ' + str(height) + ' pixels')

## basic info

In [1]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
image_path='/dds/workspace/data_ja/CAT_1_9013.jpg'
image_path='/dds/workspace/data_ja/CAT_1_9013.jpg.png'
# image_path='/dds/workspace/data_ja/test.jpg'
image_path='/dds/workspace/data_ja/test.jpg.png'
image_path='/dds/workspace/data_ja/test.jpg.2.png'
image_path='/dds/workspace/data_ja/test.jpg.3.png'

### crop test

In [ ]:
i = Image.open(image_path)
left = 100
top = 100
width = 150
height = 150
box = (left, top, left+width, top+height)
area = i.crop(box)
area

### tile test

In [2]:
 image_path='/dds/workspace/data_ja/test.jpg'
i = Image.open(image_path)
print("tile : ", i.tile)

tile :  [('jpeg', (0, 0, 960, 230), 0, ('RGB', ''))]


In [ ]:
i

In [5]:
i=None
i = Image.open(image_path)
w=560
h=130
#i.size = (w, h)
#i.tile = [('jpeg', (0, 0, 960, 230), 0, ('RGB', ''))]
# i.load()

In [ ]:
i

In [6]:
i.load()

In [ ]:
i